In [ ]:
import pandas as pd
import akimbo.pandas
import numba
import awkward as ak
import numpy as np

In [ ]:
# from https://repo.anaconda.com/pkgs/main/linux-64/repodata.json
df = pd.read_parquet("repodata.parquet", dtype_backend="pyarrow")
df

In [ ]:
df.dtypes

In [ ]:
# normal dataframe things - could groupby here
df.license_family.str.upper().value_counts()

In [ ]:
df.app_cli_opts[~df.app_cli_opts.isnull()][:2].tolist()

In [ ]:
df.depends[:1].tolist()  # a single row

In [ ]:
def discr(arr):
    return  ak.any(np.logical_and(
        arr.depends.name == "numpy", 
        ak.firsts(arr.depends.versions, axis=-1) == 2), axis=-1)

%timeit df.ak.apply(discr).sum()
df.ak.apply(discr).sum()

In [ ]:
df.ak.array[discr(df.ak.array)][:1].tolist()

In [ ]:
def num_numpy2(arr):
    """How many packages mention numpy V2 explicitly"""
    count = 0
    for package in arr:
        for depends in package:
            if (
                    depends is not None
                    and depends["name"] == "numpy"
                    and len(depends["versions"]) > 0
                    and depends["versions"][0] == 2
            ):
                count += 1
    return count


np2 = numba.njit(num_numpy2)


In [ ]:
num_numpy2(df.depends)  # same as .apply or filter/map/sum

In [ ]:
df.depends.ak.apply(np2)

In [ ]:
%timeit num_numpy2(df.depends)
%timeit df.depends.ak.apply(np2)

In [ ]:
df2 = df.explode("depends").depends.apply(pd.Series).rename(columns={"name": "dep_name"})
df2

In [ ]:
df3 = df.join(df2).explode("versions")

In [ ]:
df3

In [ ]:
print("original", df.memory_usage(deep=True).sum() // 1e6, "MB")
print("exploded", df3.memory_usage(deep=True).sum() // 1e6, "MB")

In [ ]:
%timeit df3[df3.dep_name == "numpy"].groupby(["sha256"]).versions.first().value_counts()
df3[df3.dep_name == "numpy"].groupby(["sha256"]).versions.first().value_counts()